<a href="https://colab.research.google.com/github/mrland99/helix-sandbox/blob/master/oxdna_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from psutil import virtual_memory
from os import cpu_count
colab_pro = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(colab_pro))
print('Your runtime has {} cpus.'.format(cpu_count()))

Your runtime has 37.8 gigabytes of available RAM

Your runtime has 40 cpus.


In [ ]:
# Install 
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2509k  100 2509k    0     0  16.1M      0 --:--:-- --:--:-- --:--:-- 16.0M
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


In [ ]:
# Check python version
import sys
print(sys.version)
!python3 --version
!python --version

3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]
Python 3.8.16
Python 3.8.16


In [ ]:
# Installing oxDNA as a seperate folder. 
!git clone https://github.com/lorenzo-rovigatti/oxDNA
%cd oxDNA
%mkdir build
%cd build
!cmake .. -DPython=ON -DOxpySystemInstall=On  #dropping the -DCUDA=ON to make it build a lot faster. Use the flag while using GPU in runtime.
!make -j4
!make install
%time

In [ ]:
# Check if oxDNA Analysis Tool (OAT) works?
!oat config

In [ ]:
# Correcting numpy binary incompatibility!
!python -m pip install /content/oxDNA/analysis --no-build-isolation

In [ ]:
!make test_oxpy

In [ ]:
# Append PATH for OAT 
import sys
sys.path.append("/usr/local/lib/python3.8/dist-packages")  

In [ ]:
! echo $PYTHONPATH
%env PYTHONPATH="$/env/python:/usr/local/lib/python3.9/dist-packages/"
! echo $PYTHONPATH

/env/python
env: PYTHONPATH="$/env/python:/usr/local/lib/python3.9/dist-packages/"
"$/env/python:/usr/local/lib/python3.9/dist-packages/"


In [ ]:
# Mount KanoTX Drive, and Shared Folder for Simulation Data
from google.colab import drive
drive.mount('/content/drive')
     

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Getting Code Execution Time
!pip install ipython-autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%load_ext autotime

time: 319 µs (started: 2022-12-18 22:35:24 +00:00)


# Imports

In [ ]:
from typing import List
import numpy as np
from os import path
from sys import stderr
from dataclasses import dataclass
from collections import namedtuple
import oxpy
#from oxDNA_analysis_tools.UTILS.oat_multiprocesser import oat_multiprocesser : TO get multiprocessor to work
from oxDNA_analysis_tools.UTILS.data_structures import Monomer, TopInfo, TrajInfo
from oxDNA_analysis_tools.UTILS.RyeReader import describe, strand_describe, get_input_parameter
import pandas as pd
import time

# all functions required to read a configuration using the new RyeReader
from oxDNA_analysis_tools.UTILS.RyeReader import describe, get_confs, inbox

In [ ]:
import os, fnmatch
import glob

def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

parent_dir = '/content/drive/MyDrive/Kano Projects/02-Project Noodle Theory/05 - Shared Data/02-Simulations/CO'
samples = glob.glob(f"{parent_dir}*/")

path_df = pd.DataFrame(index = ['top_path', 'traj_path', 'input_path'])
for s in samples:
    top_path = find('*.top', s)[0]
    traj_path = find('*.dat', s)[0]
    input_path = find('*input*', s)[0]
    path_df[s.split('/')[-2]] = [top_path, traj_path, input_path]
path_df

,CO1-v1,CO1-v10,CO1-v11,CO1-v12,CO1-v8,CO1-v9,CO3-v10,CO3-v11,CO3-v12,CO3-v8,CO3-v9,CO5-v10,CO5-v11,CO5-v12,CO5-v8,CO5-v9
top_path,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...
traj_path,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...
input_path,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...,/content/drive/MyDrive/Kano Projects/02-Projec...


In [ ]:
c = 'CO1-v8'
top = path_df[c][0]
traj = path_df[c][1]
input_file = path_df[c][2]
top_info, traj_info = describe(top, traj)

In [ ]:
traj_info.nconfs

1

In [ ]:
RESULTS_PATH = '/content/drive/MyDrive/Kano Projects/02-Project Noodle Theory/05 - Shared Data/06-AnalysisResults/'
for s in samples:
  new_dir = RESULTS_PATH + s.split('/')[-2]
  if not os.path.isdir(new_dir):
    os.mkdir(new_dir)

CO1-v1
CO1-v10
CO1-v11
CO1-v12
CO1-v8
CO1-v9
CO3-v10
CO3-v11
CO3-v12
CO3-v8
CO3-v9
CO5-v10
CO5-v11
CO5-v12
CO5-v8
CO5-v9
time: 8.84 ms (started: 2022-12-15 16:40:38 +00:00)


# **Define Analysis Functions**

## Get all paired nucleotides

In [ ]:
from operator import itemgetter
from itertools import groupby

def get_all_pairs(top, traj, input_file):
  # RyeReader uses indexing allows for random access in the trajectory
  top_info, traj_info = describe(top, traj)
  sys, monomer = strand_describe(top)

  # A compute context is a single variable (in this case a namedtuple, but could also be a dict or a dataclass)
  # which contains the arguments needed for a parallelized function.
  # We use this style to avoid having to pass variable numbers of arguments to the parallelizer.
  ComputeContext = namedtuple("ComputeContext",["traj_info",
                                                "top_info",
                                                "input_file",
                                                "monomers"])

  ctx = ComputeContext(traj_info, top_info, input_file, monomer)

  all_pairs = []
  with oxpy.Context():
      inp = oxpy.InputFile()
      inp.init_from_filename(ctx.input_file)
      inp["list_type"] = "cells"
      inp["trajectory_file"] = ctx.traj_info.path
      inp["topology"] = ctx.top_info.path
      inp["backend"] = "CPU"
      inp["backend_precision"] = "double"
      #inp["max_backbone_force"] = "1"
      inp["use_average_seq"] = "1"
      inp["analysis_data_output_1"] = '{ \n name = stdout \n print_every = 1e10 \n col_1 = { \n id = my_obs \n type = hb_list \n } \n }'

      # if (not inp["use_average_seq"] or inp.get_bool("use_average_seq")) and "RNA" in inp["interaction_type"]:
      # print("WARNING: Sequence dependence not set for RNA model, wobble base pairs will be ignored", file=stderr)

      backend = oxpy.analysis.AnalysisBackend(inp)
      
      while backend.read_next_configuration():
              pairs = backend.config_info().get_observable_by_id("my_obs").get_output_string(backend.config_info().current_step).strip().split('\n')
              all_pairs.append(pairs)  # Store all paired nucleotides in all_pairs
      return all_pairs

time: 2.8 ms (started: 2022-12-19 01:20:02 +00:00)


# Create duplex dataframe

In [ ]:
def create_duplex_df(pairs):
  tp = 0
  rows = [] # rows for all duplex_df

  for pairs in all_pairs:
      temp = pairs[1:]
      
      # convert from list of strings to two columns in a dataframe
      a = []
      b = []
      for p in temp:
          p = p.split()
          a.append(int(p[0]))
          b.append(int(p[1]))

      df = pd.DataFrame([a, b], index = ['a', 'b']).T
      df = df.sort_values('a')
      df.index = range(df.shape[0])
      
      # get start and stop row indexes of consecutive values in df['a']
      data = df['a']
      ranges = []
      for k,g in groupby(enumerate(data),lambda x:x[0]-x[1]):
          # if 0, returns index.
          group = (map(itemgetter(0),g))
          group = list(map(int,group))
          ranges.append((group[0],group[-1]))
      
      # for each consecutive range for df['a'], split into consecutive range for df['b']
      ranges_final = []
      for r in ranges:
          idx_1, idx_2 = r
          lst = df['b'][idx_1: idx_2 + 1][::-1] # since b is decreasing, convert decreasing to increasing lst 

          for k,g in groupby(enumerate(lst),lambda x:x[0]-x[1]):
              # if 0, returns index.
              group = (map(itemgetter(0),g))
              group = list(map(int,group))
              # we map index back to that of the original indexes of df
              ranges_final.append((idx_2 - group[-1], idx_2 - group[0]))

      # sort by decreasing order for readability (once we convert to 5' to 3' it will be increasing)
      ranges_final = sorted(ranges_final, key=lambda r: r[0], reverse=True)
          
      for r in ranges_final:
          # only count duplexes with 3 consecutive pairs
          if r[1]- r[0] > 1:
              start_1 = df['a'][r[0]]
              end_1 = df['a'][r[1]]
              start_2 = df['b'][r[1]]
              end_2 = df['b'][r[0]]
              rows.append([tp, start_1, end_1, start_2, end_2])
      
      tp += 1

  reverse_duplex_df = pd.DataFrame(rows, columns = ['time', 'start_1', 'end_1', 'start_2', 'end_2'])

  duplex_df = reverse_duplex_df.copy()
  seq_len = top_info.nbases
  # remember oxDNA is 0-indexed
  duplex_df['start_1'] = seq_len - duplex_df['end_2'] - 1
  duplex_df['end_1'] = seq_len - duplex_df['start_2'] - 1
  duplex_df['start_2'] = seq_len - duplex_df['end_1'] - 1
  duplex_df['end_2'] = seq_len - duplex_df['start_1'] - 1
  return duplex_df

time: 4.15 ms (started: 2022-12-15 16:52:03 +00:00)


# Calculate Distances

In [ ]:
from oxDNA_analysis_tools.distance import distance

time: 531 µs (started: 2022-12-18 23:14:02 +00:00)


In [ ]:
df = pd.read_excel(SEQUENCES_PATH, index_col = 0)

time: 291 ms (started: 2022-12-18 23:03:31 +00:00)


In [ ]:
seq_df = df[df['Name'] == 'CO1-v1']


time: 1.51 ms (started: 2022-12-18 23:02:40 +00:00)


In [ ]:
seq_df['5\' HA'].isnull().values[0]

True

time: 3.1 ms (started: 2022-12-18 23:06:11 +00:00)


In [ ]:
def get_distances(sim_name, top, traj, input_file, sequence_path):
  # RyeReader uses indexing allows for random access in the trajectory
  top_info, traj_info = describe(top, traj)
  sys, monomer = strand_describe(top)
  df = pd.read_excel(sequence_path, index_col = 0)
  seq_df = df[df['Name'] == sim_name]

  total_seq = seq_df['Total sequence (cssDNA or lssDNA)'].values[0].upper()
  five_ha_seq = seq_df['5\' HA'].values[0].upper()
  three_ha_seq = seq_df['3\' HA'].values[0].upper()
  backbone_seq = seq_df['Backbone seq fragment'].values[0].upper()

  # Get index in 5'-> 3' orientation since default is in 3' -> 5'
  five_start = total_seq.find(five_ha_seq)
  three_start = total_seq.find(three_ha_seq)
  five_end = total_seq.find(five_ha_seq) + len(five_ha_seq) - 1
  three_end = total_seq.find(three_ha_seq) + len(three_ha_seq) - 1
  backbone_start = total_seq.find(backbone_seq)
  backbone_end = total_seq.find(backbone_seq) + len(backbone_seq) - 1

  seq_len = top_info.nbases
  result = distance_2(traj_info, top_info, 
                    [five_end, five_start, five_start, three_start, backbone_start], 
                    [three_start, three_end, five_end, three_end, backbone_end], 
                    seq_len)
  
  return result

# takes in 5' to 3' indexes
def distance_2(traj_info, top_info, lst1, lst2, seq_len):
    
    # Convert 5'->3' index to 3'->5' for oxdna
    for i in range(len(lst1)):
        lst1[i] = seq_len - lst1[i] - 1
    for i in range(len(lst2)):
        lst2[i] = seq_len - lst2[i] - 1
        
    return distance([traj_info], [top_info], [lst2],[lst1])

time: 2.9 ms (started: 2022-12-18 23:14:05 +00:00)


# Calculate number of times a nucleotide changes pairing partner

In [ ]:
def get_num_pair_partners(pairs, seq_len):
  # initialize pairing vectors
  totals = np.zeros((seq_len))
  past_pair = np.full((seq_len), -1)

  for pairs_at_single_tp in pairs:
      # initialize pairing vetor for current time point
      curr_pair = np.full((seq_len), -1)
      for p in pairs_at_single_tp[1:]:
          idx1 = int(p.split(' ')[0])
          idx2 = int(p.split(' ')[1])
          curr_pair[idx1] = idx2
          curr_pair[idx2] = idx1
      # calculate if nucleotide changes pairing partner    
      diffs = (past_pair != curr_pair).astype(int)
      # update 
      totals = totals + diffs
      past_pair = curr_pair

  # Flip order so result is stored 5' -> 3'
  return np.flip(totals)

time: 1.5 ms (started: 2022-12-19 01:01:58 +00:00)


# **Generate Analysis**

In [ ]:
RESULTS_PATH = '/content/drive/MyDrive/Kano Projects/02-Project Noodle Theory/05 - Shared Data/06-AnalysisResults/'
SEQUENCES_PATH = '/content/drive/MyDrive/Kano Projects/02-Project Noodle Theory/05 - Shared Data/01-Sequences/Noodle-project-sequence-collection.xlsx'

time: 469 µs (started: 2022-12-18 23:14:10 +00:00)


In [ ]:
runtimes = []
for c in path_df.columns:
  print(c)
  top = path_df[c][0]
  traj = path_df[c][1]
  input_file = path_df[c][2]

  start_time = time.time()
  #pairs = get_all_pairs(top, traj, input_file)
  #df = create_duplex_df(pairs)
  # runtimes.append(time.time() - start_time)

  # df.to_csv(RESULTS_PATH + c + '/duplex_df.csv')

CO1-v1
CO1-v10
CO1-v11
CO1-v12
CO1-v8
CO1-v9
CO3-v10
CO3-v11
CO3-v12
CO3-v8
CO3-v9
CO5-v10
CO5-v11
CO5-v12
CO5-v8
CO5-v9
time: 2.8 ms (started: 2022-12-19 01:03:51 +00:00)


Number of times nucleotide changes pairs

In [ ]:
runtimes = []
changes = []
for c in path_df.columns:
  print(c)
  top = path_df[c][0]
  traj = path_df[c][1]
  input_file = path_df[c][2]
  top_info, traj_info = describe(top, traj)
  seq_len = top_info.nbases

  start_time = time.time()
  pairs = get_all_pairs(top, traj, input_file)
  changes.append(get_num_pair_partners(pairs, seq_len))
  runtimes.append(time.time() - start_time)


CO1-v1
CO1-v10
CO1-v11
CO1-v12
CO1-v8
CO1-v9
CO3-v10
CO3-v11
CO3-v12
CO3-v8
CO3-v9
CO5-v10
CO5-v11
CO5-v12
CO5-v8
CO5-v9
time: 15.4 s (started: 2022-12-19 01:03:55 +00:00)


In [ ]:
c = 'CO1-v8'
top = path_df[c][0]
traj = path_df[c][1]
input_file = path_df[c][2]

top_info, traj_info = describe(top, traj)
seq_len = top_info.nbases
pairs = get_all_pairs(top, traj, input_file)


# initialize pairing vectors
totals = np.zeros((seq_len))
past_pair = np.full((seq_len), -1)

for pairs_at_single_tp in pairs:
  # initialize pairing vetor for current time point
  curr_pair = np.full((seq_len), -1)
  for p in pairs_at_single_tp[1:]:
      idx1 = int(p.split(' ')[0])
      idx2 = int(p.split(' ')[1])
      curr_pair[idx1] = idx2
      curr_pair[idx2] = idx1
  # calculate if nucleotide changes pairing partner    
  diffs = (past_pair != curr_pair).astype(int)
  # update 
  print(diffs)
  totals = totals + diffs
  past_pair = curr_pair

[1 0 0 ... 1 1 1]
time: 44.8 ms (started: 2022-12-19 01:16:52 +00:00)


Distances

In [ ]:
c = 'CO1-v8'
top = path_df[c][0]
traj = path_df[c][1]
input_file = path_df[c][2]

results = get_distances(c, top, traj, input_file, SEQUENCES_PATH)

INFO: Working on trajectory: /content/drive/MyDrive/Kano Projects/02-Project Noodle Theory/05 - Shared Data/02-Simulations/CO1-v8/CO1-v8_e.dat


Starting up 1 processes for 1 chunks
All spawned, waiting for results
time: 229 ms (started: 2022-12-18 23:14:23 +00:00)


INFO: Processing in blocks of 20 configurations
INFO: You can modify this number by running oat config -n <number>, which will be persistent between analyses.


In [ ]:
top

'/content/drive/MyDrive/Kano Projects/02-Project Noodle Theory/05 - Shared Data/02-Simulations/CO1-v8/CO1-v8.top'

time: 2.63 ms (started: 2022-12-18 23:16:15 +00:00)


In [ ]:
# runtimes_df = pd.DataFrame(runtimes, index = path_df.columns, columns = ['Create duplex df'])

time: 1.17 ms (started: 2022-12-15 17:51:27 +00:00)


In [ ]:
# runtimes_df.to_csv(RESULTS_PATH + 'runtimes.csv')

time: 6.15 ms (started: 2022-12-15 17:51:53 +00:00)


In [ ]:
import oxDNA_analysis_tools 
oxDNA_analysis_tools.__version__

'2.0.6'

time: 2.97 ms (started: 2022-12-18 23:19:42 +00:00)


In [ ]:
import sys
sys.version

'3.8.16 (default, Dec  7 2022, 01:12:13) \n[GCC 7.5.0]'

time: 2.61 ms (started: 2022-12-18 23:25:57 +00:00)
